In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from common.db_config import db_config
from tqdm.notebook import tqdm
from common.data_processor import *
from common.utils import split_data
import optuna.integration.lightgbm as lgb_o
import lightgbm as lgb

In [9]:
r = Results.read_db(db_config['main'], begin_date=20150101, end_date=20211231, flat_only=True)
r.data_p.head()

,horse_no,frame_no,horse_id,impost,jockey_id,trainer_id,date,place_id,hold_no,hold_day,...,ground,weather,horse_num,month,rank,sex,age,weight,weight_change,win_prise
race_id,,,,,,,,,,,,,,,,,,,,,
201601010101,6,6,2014102322,52.0,01151,01024,2016-07-30,1,1,1,...,良,曇,10,20,1,牝,2,434,-4,500.0
201601010101,2,2,2014105971,54.0,00700,01066,2016-07-30,1,1,1,...,良,曇,10,20,1,牡,2,438,-2,500.0
201601010101,5,5,2014100723,54.0,01091,00359,2016-07-30,1,1,1,...,良,曇,10,20,1,牡,2,458,-2,500.0
201601010101,4,4,2014105192,51.0,01163,01046,2016-07-30,1,1,1,...,良,曇,10,20,0,牡,2,500,-4,500.0
201601010101,3,3,2014106056,54.0,01014,01058,2016-07-30,1,1,1,...,良,曇,10,20,0,牡,2,468,2,500.0


In [10]:
hr = HorseResults.read_db(db_config['main'])
hr.data_p.head()

,race_id,date,place_id,weather,race_no,horse_no,win_odds,popularity,arriving_order,ground,goal_time,race_type,distance,ground,time_diff,corner_pass,last_three_furlong,prise,first_corner,last_corner
horse_id,,,,,,,,,,,,,,,,,,,,
2005105204,201605050508,2016-11-19,5,小雨,8,1,26.1,6.0,12.0,稍,219.1,障害,3110,稍,4.4,3-3-5-6,14.1,0.0,3.0,6.0
2005105204,201604030104,2016-10-15,4,晴,4,13,3.0,1.0,1.0,良,186.2,障害,2850,良,-1.2,2-2-1-1,13.1,750.0,2.0,1.0
2005105204,201604010301,2016-05-07,4,雨,1,12,2.8,1.0,4.0,稍,193.1,障害,2890,稍,1.7,1-1-1-1,13.4,110.0,1.0,1.0
2005105204,201603010204,2016-04-10,3,晴,4,1,2.4,1.0,4.0,良,180.5,障害,2750,良,0.9,7-6-5-5,13.1,110.0,7.0,5.0
2005105204,201607020205,2016-03-13,7,曇,5,1,6.9,3.0,2.0,良,197.8,障害,3000,良,0.1,9-8-5-2,13.2,300.0,9.0,2.0


In [11]:
r.merge_horse_results(hr, ave_samples=5)
r.data_m.head()

100%|██████████| 652/652 [04:23<00:00,  2.47it/s]


,horse_no,frame_no,horse_id,impost,jockey_id,trainer_id,date,place_id,hold_no,hold_day,...,win_prise,l_days,arriving_order_5R,popularity_5R,distance_5R,time_diff_5R,last_three_furlong_5R,first_corner_5R,last_corner_5R,prise_5R
race_id,,,,,,,,,,,,,,,,,,,,,
201601010101,6,6,2014102322,52.0,01151,01024,2016-07-30,1,1,1,...,500.0,14.0,8.0,10.5,1800.0,1.25,35.75,13.0,10.5,0.0
201601010101,2,2,2014105971,54.0,00700,01066,2016-07-30,1,1,1,...,500.0,27.0,7.0,3.0,1600.0,1.30,36.30,6.0,8.0,0.0
201601010101,5,5,2014100723,54.0,01091,00359,2016-07-30,1,1,1,...,500.0,20.0,4.5,3.5,1100.0,1.05,35.90,4.5,4.5,65.0
201601010101,4,4,2014105192,51.0,01163,01046,2016-07-30,1,1,1,...,500.0,13.0,4.0,6.0,1800.0,0.50,35.00,2.0,5.0,110.0
201601010101,3,3,2014106056,54.0,01014,01058,2016-07-30,1,1,1,...,500.0,34.0,2.0,3.0,1800.0,0.50,35.40,9.0,10.0,280.0


In [12]:
r.data_m.to_pickle('./results_m_2015_2021.pickle')

In [13]:
p = Peds.read_db(db_config['main'])
p.data_e

,father,mother,fathers_father,fathers_mother,mothers_father,mothers_mother
id,,,,,,
2018105875,12451,7243,9673,10984,5084,11540
2018105258,6462,17241,10432,10492,13198,16984
2018101134,18413,9782,13198,11099,8318,15063
2018102791,2017,13748,1865,4505,1684,1096
2018101539,7475,15285,12451,12784,17420,15314
...,...,...,...,...,...,...
2019105297,7642,12953,13198,7752,3537,2190
2019100793,11023,17677,8410,19830,11930,14390
2019105870,15448,17169,9332,10133,4836,11163


In [14]:
r.merge_peds(p)
r.data_pe

,horse_no,frame_no,horse_id,impost,jockey_id,trainer_id,date,place_id,hold_no,hold_day,...,last_three_furlong_5R,first_corner_5R,last_corner_5R,prise_5R,father,mother,fathers_father,fathers_mother,mothers_father,mothers_mother
race_id,,,,,,,,,,,,,,,,,,,,,
201601010101,6,6,2014102322,52.0,01151,01024,2016-07-30,1,1,1,...,35.75,13.0,10.5,0.0,5012,15707,6694,17579,3689,4115
201601010101,2,2,2014105971,54.0,00700,01066,2016-07-30,1,1,1,...,36.30,6.0,8.0,0.0,14808,10694,923,2997,9673,15572
201601010101,5,5,2014100723,54.0,01091,00359,2016-07-30,1,1,1,...,35.90,4.5,4.5,65.0,4650,17626,1341,213,11106,11775
201601010101,4,4,2014105192,51.0,01163,01046,2016-07-30,1,1,1,...,35.00,2.0,5.0,110.0,6706,16921,4254,4178,18201,11128
201601010101,3,3,2014106056,54.0,01014,01058,2016-07-30,1,1,1,...,35.40,9.0,10.0,280.0,14771,14479,9673,4641,1573,1696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202106050912,9,7,2016104480,57.0,05386,00386,2021-12-28,6,5,9,...,35.22,8.6,8.6,220.0,14771,9404,9673,4641,3582,2684
202106050912,11,8,2017105116,57.0,05473,01141,2021-12-28,6,5,9,...,34.24,7.0,8.8,481.8,9149,12015,11183,17479,16433,16137
202106050912,7,6,2017102037,57.0,01173,01005,2021-12-28,6,5,9,...,34.16,11.0,10.6,0.0,19911,11827,13198,13424,16433,18562


In [15]:
r.process_categorical()
r.data_c

,horse_no,frame_no,horse_id,impost,jockey_id,trainer_id,date,place_id,hold_no,hold_day,...,last_three_furlong_5R,first_corner_5R,last_corner_5R,prise_5R,father,mother,fathers_father,fathers_mother,mothers_father,mothers_mother
race_id,,,,,,,,,,,,,,,,,,,,,
201601010101,6,6,9481,52.0,112,300,2016-07-30,1,1,1,...,35.75,13.0,10.5,0.0,5012,15707,6694,17579,3689,4115
201601010101,2,2,12002,54.0,11,51,2016-07-30,1,1,1,...,36.30,6.0,8.0,0.0,14808,10694,923,2997,9673,15572
201601010101,5,5,8447,54.0,63,301,2016-07-30,1,1,1,...,35.90,4.5,4.5,65.0,4650,17626,1341,213,11106,11775
201601010101,4,4,11372,51.0,124,43,2016-07-30,1,1,1,...,35.00,2.0,5.0,110.0,6706,16921,4254,4178,18201,11128
201601010101,3,3,12079,54.0,26,302,2016-07-30,1,1,1,...,35.40,9.0,10.0,280.0,14771,14479,9673,4641,1573,1696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202106050912,9,7,20253,57.0,189,345,2021-12-28,6,5,9,...,35.22,8.6,8.6,220.0,14771,9404,9673,4641,3582,2684
202106050912,11,8,25316,57.0,215,104,2021-12-28,6,5,9,...,34.24,7.0,8.8,481.8,9149,12015,11183,17479,16433,16137
202106050912,7,6,23251,57.0,134,337,2021-12-28,6,5,9,...,34.16,11.0,10.6,0.0,19911,11827,13198,13424,16433,18562


In [16]:
tmp_df = r.get_final_data(True)

In [17]:
train = tmp_df[tmp_df['date'] <= dt.datetime(2020, 12, 31)]
test = tmp_df[tmp_df['date'] >= dt.datetime(2021, 1, 1)]

In [18]:
train, valid = split_data(train)

In [19]:
X_train = train.drop(['date', 'rank'], axis=1)
y_train = train['rank']
X_valid = valid.drop(['date', 'rank'], axis=1)
y_valid = valid['rank']
X_test = test.drop(['date', 'rank'], axis=1)
y_test = test['rank']

In [20]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 148233 entries, 201608010105 to 201905030611
Data columns (total 36 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   horse_no               148233 non-null  int64  
 1   frame_no               148233 non-null  int64  
 2   impost                 148233 non-null  float64
 3   jockey_id              148233 non-null  int32  
 4   trainer_id             148233 non-null  int32  
 5   place_id               148233 non-null  int32  
 6   hold_no                148233 non-null  int64  
 7   hold_day               148233 non-null  int64  
 8   race_no                148233 non-null  int64  
 9   distance               148233 non-null  int64  
 10  race_type              148233 non-null  int64  
 11  turn                   148233 non-null  int64  
 12  ground                 148233 non-null  int64  
 13  weather                148233 non-null  int64  
 14  horse_num              1

In [21]:
lgb_o_train = lgb_o.Dataset(X_train, y_train)
lgb_o_valid = lgb_o.Dataset(X_valid, y_valid, reference=lgb_o_train)

In [22]:
params = {
    'objective': 'binary',
    'random_state': 100,
}

cat_list = ['jockey_id', 'trainer_id', 'place_id', 'race_type', 'turn', 'ground', 'weather', 'sex', 'father', 'mother', 'fathers_father', 'fathers_mother', 'mothers_father', 'mothers_mother']

lgb_clf_o = lgb_o.train(params, lgb_o_train, valid_sets=[lgb_o_train, lgb_o_valid], verbose_eval=100, early_stopping_rounds=100, categorical_feature=cat_list)

[I 2022-01-15 11:28:34,783] A new study created in memory with name: no-name-01be4411-f0d3-4b6e-bb90-0abc02a8530a
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]c:\work\KeibaAI\.venv\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\work\KeibaAI\.venv\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['father', 'fathers_father', 'fathers_mother', 'ground', 'jockey_id', 'mother', 'mothers_father', 'mothers_mother', 'place_id', 'race_type', 'sex', 'trainer_id', 'turn', 'weather']
  _log_warning('categorical_feature in Dataset is overridden.\n'
c:\work\KeibaAI\.venv\lib\site-packages\lightgbm\engine.p

[LightGBM] [Info] Number of positive: 31528, number of negative: 116705
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016755 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19630
[LightGBM] [Info] Number of data points in the train set: 148233, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.212692 -> initscore=-1.308773
[LightGBM] [Info] Start training from score -1.308773
Training until validation scores don't improve for 100 rounds


c:\work\KeibaAI\.venv\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
c:\work\KeibaAI\.venv\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[100]	valid_0's binary_logloss: 0.344989	valid_1's binary_logloss: 0.474002


feature_fraction, val_score: 0.467721:  14%|#4        | 1/7 [00:03<00:18,  3.16s/it]

...

[LightGBM] [Info] Number of positive: 31528, number of negative: 116705
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011959 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19630
[LightGBM] [Info] Number of data points in the train set: 148233, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.212692 -> initscore=-1.308773
[LightGBM] [Info] Start training from score -1.308773
Training until validation scores don't improve for 100 rounds


c:\work\KeibaAI\.venv\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
c:\work\KeibaAI\.venv\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[100]	valid_0's binary_logloss: 0.431847	valid_1's binary_logloss: 0.464397
[200]	valid_0's binary_logloss: 0.417907	valid_1's binary_logloss: 0.463286


min_data_in_leaf, val_score: 0.463007: 100%|##########| 5/5 [00:11<00:00,  2.29s/it]

[300]	valid_0's binary_logloss: 0.408385	valid_1's binary_logloss: 0.463404
Early stopping, best iteration is:
[215]	valid_0's binary_logloss: 0.416225	valid_1's binary_logloss: 0.463007


In [97]:
lgb_clf_o.params

{'objective': 'binary',
 'random_state': 100,
 'feature_pre_filter': False,
 'lambda_l1': 9.927931426029202,
 'lambda_l2': 2.2847450729790525e-08,
 'num_leaves': 5,
 'feature_fraction': 0.4,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 20,
 'num_iterations': 1000,
 'early_stopping_round': 100,
 'categorical_column': [3, 4, 5, 10, 11, 12, 13, 16, 30, 31, 32, 33, 34, 35]}

In [100]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_valid = lgb.Dataset(X_test, y_test)

model = lgb.train(lgb_clf_o.params, lgb_train, valid_sets=lgb_valid, verbose_eval=50)

c:\work\KeibaAI\.venv\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\work\KeibaAI\.venv\lib\site-packages\lightgbm\engine.py:240: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 31528, number of negative: 116705
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009872 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19630
[LightGBM] [Info] Number of data points in the train set: 148233, number of used features: 36
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.212692 -> initscore=-1.308773
[LightGBM] [Info] Start training from score -1.308773
Training until validation scores don't improve for 100 rounds
[50]	valid_0's binary_logloss: 0.466794
[100]	valid_0's binary_logloss: 0.464787
[150]	valid_0's binary_logloss: 0.464264
[200]	valid_0's binary_logloss: 0.464616
[250]	valid_0's binary_logloss: 0.464923
Early stopping, best iteration is:
[179]	valid_0's binary_logloss: 0.463945


In [105]:
y_pred_train = model.predict(X_train, num_iteration=model.best_iteration)
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

In [106]:
from sklearn.metrics import roc_curve, roc_auc_score

In [107]:
roc_auc_score(y_train, y_pred_train)

0.7953444506668292

In [108]:
roc_auc_score(y_test, y_pred)

0.7358713485851867

In [109]:
importances = pd.DataFrame(model.feature_importance(), index=X_train.columns, columns=['importance'])
importances.sort_values('importance', ascending=False)[:20]

,importance
mother,143
mothers_mother,129
win_prise,50
trainer_id,47
jockey_id,46
mothers_father,43
horse_num,26
father,24
arriving_order_5R,21
fathers_mother,20


In [10]:
rc = RaceCard.scrape(['202206010502', '202207010502', '202207010503', '202210010102', '202210010103'], 20220115)
rc.data_p

,horse_no,frame_no,horse_id,impost,jockey_id,trainer_id,date,place_id,hold_no,hold_day,...,turn,ground,weather,horse_num,month,sex,age,weight,weight_change,win_prise
race_id,,,,,,,,,,,,,,,,,,,,,
202206010502,1,1,2019100836,53.0,01192,01031,2022-01-15,6,1,5,...,右,稍,晴,16,20,牡,3,506,-6,520
202206010502,2,1,2019104854,56.0,05386,01103,2022-01-15,6,1,5,...,右,稍,晴,16,20,牡,3,542,0,520
202206010502,3,2,2019102082,55.0,01167,01048,2022-01-15,6,1,5,...,右,稍,晴,16,20,牡,3,518,0,520
202206010502,4,2,2019100328,56.0,01029,00392,2022-01-15,6,1,5,...,右,稍,晴,16,20,牡,3,482,2,520
202206010502,5,3,2019103586,56.0,01161,01115,2022-01-15,6,1,5,...,右,稍,晴,16,20,牡,3,494,4,520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202210010103,10,6,2019101260,56.0,01095,01039,2022-01-15,10,1,1,...,右,良,晴,14,20,牡,3,476,0,520
202210010103,11,7,2019104257,54.0,01093,01104,2022-01-15,10,1,1,...,右,良,晴,14,20,牝,3,408,-12,520
202210010103,12,7,2019104333,54.0,01178,00425,2022-01-15,10,1,1,...,右,良,晴,14,20,牝,3,458,8,520


In [12]:
rc.merge_horse_results(hr, ave_samples=5)
rc.merge_peds(p)

  0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
r.data_p.head()

,horse_no,frame_no,horse_id,impost,jockey_id,trainer_id,date,place_id,hold_no,hold_day,...,ground,weather,horse_num,month,rank,sex,age,weight,weight_change,win_prise
race_id,,,,,,,,,,,,,,,,,,,,,
201601010101,6,6,2014102322,52.0,01151,01024,2016-07-30,1,1,1,...,良,曇,10,20,1,牝,2,434,-4,500.0
201601010101,2,2,2014105971,54.0,00700,01066,2016-07-30,1,1,1,...,良,曇,10,20,1,牡,2,438,-2,500.0
201601010101,5,5,2014100723,54.0,01091,00359,2016-07-30,1,1,1,...,良,曇,10,20,1,牡,2,458,-2,500.0
201601010101,4,4,2014105192,51.0,01163,01046,2016-07-30,1,1,1,...,良,曇,10,20,0,牡,2,500,-4,500.0
201601010101,3,3,2014106056,54.0,01014,01058,2016-07-30,1,1,1,...,良,曇,10,20,0,牡,2,468,2,500.0


In [17]:
rc.process_categorical(r)
rc.data_c

,horse_no,frame_no,horse_id,impost,jockey_id,trainer_id,date,place_id,hold_no,hold_day,...,last_three_furlong_5R,first_corner_5R,last_corner_5R,prise_5R,father,mother,fathers_father,fathers_mother,mothers_father,mothers_mother
race_id,,,,,,,,,,,,,,,,,,,,,
202206010502,1,1,31799,53.0,153,83,2022-01-15,6,1,5,...,39.600000,12.500000,12.500000,0.0,17006,11576,17007,2464,1699,9000
202206010502,2,1,33373,56.0,189,151,2022-01-15,6,1,5,...,42.200000,3.000000,4.000000,0.0,7668,15193,16493,10143,88,3121
202206010502,3,2,32260,55.0,128,97,2022-01-15,6,1,5,...,38.600000,7.000000,8.000000,0.0,13633,17771,10659,6730,12451,10943
202206010502,4,2,31581,56.0,33,21,2022-01-15,6,1,5,...,39.740000,11.400000,10.800000,0.0,13871,19841,1480,1162,4744,4710
202206010502,5,3,32813,56.0,122,163,2022-01-15,6,1,5,...,35.933333,1.666667,1.333333,160.0,13753,5392,7905,4964,10432,12063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202210010103,10,6,31942,56.0,66,89,2022-01-15,10,1,1,...,37.560000,3.400000,2.400000,142.0,17006,18083,17007,2464,9332,20911
202210010103,11,7,33090,54.0,65,152,2022-01-15,10,1,1,...,38.200000,7.000000,6.000000,0.0,18808,18689,11930,13903,18741,7891
202210010103,12,7,33115,54.0,139,41,2022-01-15,10,1,1,...,35.700000,2.800000,2.800000,70.8,9787,7981,4367,1594,8410,6855


In [23]:
r.data_p.info()

<class 'pandas.core.frame.DataFrame'>
Index: 276689 entries, 201601010101 to 202110040812
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   horse_no       276689 non-null  int64         
 1   frame_no       276689 non-null  int64         
 2   horse_id       276689 non-null  object        
 3   impost         276689 non-null  float64       
 4   jockey_id      276689 non-null  object        
 5   trainer_id     276689 non-null  object        
 6   date           276689 non-null  datetime64[ns]
 7   place_id       276689 non-null  object        
 8   hold_no        276689 non-null  int64         
 9   hold_day       276689 non-null  int64         
 10  race_no        276689 non-null  int64         
 11  distance       276689 non-null  int64         
 12  race_type      276689 non-null  object        
 13  turn           276689 non-null  object        
 14  ground         276689 non-null  object  

In [22]:
rc.data_p.info()

<class 'pandas.core.frame.DataFrame'>
Index: 70 entries, 202206010502 to 202210010103
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   horse_no       70 non-null     object        
 1   frame_no       70 non-null     object        
 2   horse_id       70 non-null     object        
 3   impost         70 non-null     object        
 4   jockey_id      70 non-null     object        
 5   trainer_id     70 non-null     object        
 6   date           70 non-null     datetime64[ns]
 7   place_id       70 non-null     int64         
 8   hold_no        70 non-null     int64         
 9   hold_day       70 non-null     int64         
 10  race_no        70 non-null     int64         
 11  distance       70 non-null     int64         
 12  race_type      70 non-null     object        
 13  turn           70 non-null     object        
 14  ground         70 non-null     object        
 15  weather  